# Web Scraping Medium articles with tags ML/DL/AI using BS & SELENIUM

In [1]:
#import all libraries
import urllib.request
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
#save the links we wanna scrape
arificial_intelligence = "https://medium.com/tag/product-management"


In [3]:
!pip install selenium
from selenium import webdriver

### Dynamic loading with selenium

In [4]:
driver = webdriver.Chrome('C:/Software Projects/chromedriver_win32/chromedriver.exe')
driver.get(arificial_intelligence)

C:\Users\annas\AppData\Local\Temp\ipykernel_22136\200098488.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Software Projects/chromedriver_win32/chromedriver.exe')


### Load the webpage that is displayed till you reach the end to collect all tagged articles

In [5]:
res = driver.execute_script("return document.documentElement.outerHTML")
print(res)
driver.quit()

<html lang="en" data-rh="lang"><head><title>The most insightful stories about Product Management - Medium</title><meta data-rh="true" charset="utf-8"><meta data-rh="true" name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1,maximum-scale=1"><meta data-rh="true" name="theme-color" content="#000000"><meta data-rh="true" name="twitter:app:name:iphone" content="Medium"><meta data-rh="true" name="twitter:app:id:iphone" content="828256236"><meta data-rh="true" property="al:ios:app_name" content="Medium"><meta data-rh="true" property="al:ios:app_store_id" content="828256236"><meta data-rh="true" property="al:android:package" content="com.medium.reader"><meta data-rh="true" property="al:android:app_name" content="Medium"><meta data-rh="true" property="fb:app_id" content="542599432471018"><meta data-rh="true" property="og:site_name" content="Medium"><meta data-rh="true" name="title" content="The most insightful stories about Product Management - Medium"><meta data-rh="tru

In [6]:
!pip install BeautifulSoup4
!pip install lxml
from bs4 import BeautifulSoup

soup = BeautifulSoup(res, 'lxml')

In [ ]:
print(soup.prettify())

In [ ]:
b = soup.find_all('div')

In [9]:
[type(item) for item in list(soup.children)]


[bs4.element.Tag]

# Articles tagged in ARTIFICIAL INTELLIGENCE

In [10]:
# Input all work data into the array
work=[]
for my_tag in soup.find_all(class_="bu of rn ro rp rq rr rs rt ru rv rw rx ry rz sa sb sc sd se sf sg sh si sj sk sl cs pc pd pf ph bx"):
    work.append(my_tag.text)

In [11]:
print(work)

[]


In [12]:
para=[]
for my_tag in soup.find_all(True,{'class':['graf graf--h4 graf-after--h3 graf--trailing graf--subtitle','graf graf--p graf-after--figure','graf graf--p graf-after--h3 graf--trailing']}):
    para.append(my_tag.text)

In [13]:
len(para)

0

In [470]:
title=[]
for my_tag in soup.find_all(True,{'class':['graf graf--h3 graf-after--figure graf--title','graf graf--h3 graf--leading graf--title','graf graf--h3 graf-after--figure graf--trailing graf--title']}):
    title.append(my_tag.text)

In [543]:
title[0]

'GAME-CHANGING TRENDS TO LOOK OUT FOR WITH\xa0AI'

In [471]:
read=[]
for my_tag in soup.find_all(class_="readingTime"):
    read.append(my_tag.get('title'))

### Input the number of votes, name, date and other information into seperate arrays

In [472]:
upvotes=[]
for my_tag in soup.find_all('span',{'class':'u-textAlignCenter u-relative u-background js-actionMultirecommendCount u-marginLeft5'}):
    upvotes.append(my_tag.text)

In [473]:
date=[]
for my_tag in soup.find_all('time'):
    date.append(my_tag.text)

In [474]:
name = []
for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken"):
    name.append(my_tag.text)

In [550]:
content=[]
alllink=[]
for my_tag in soup.find_all(class_="postArticle postArticle--short js-postArticle js-trackedPost"):
        content.append(my_tag.text)
        for link in my_tag.find_all('a'):
            alllink.append(link.get('href'))

In [551]:
for i in range(0,len(content)):
    content[i]=content[i].replace(name[i],'')
    content[i]=content[i].replace(date[i],'')
    content[i]=content[i].replace(upvotes[i],'')
    content[i]=content[i].replace(title[i],'')
    content[i]=content[i].replace('…Read more…','*')
    content[i]=content[i].replace('Read more…','*')
    content[i]=content[i].replace('\xa0',' ')
    content[i]=content[i].replace(' in ', '')
    for tag in work:
        content[i] = content[i].replace(tag, '')
    content[i]=content[i].split('*')

In [552]:
len(content)

50

In [553]:
body=[]
for i in range(0,len(content)):
    body.append(content[i][0])

In [554]:
len(body)

50

### Check if all arrays have an equal length of 50 (There are 50 articles loaded)

In [524]:
res =[]
for i in range(0,len(content)):
    if (len(content[i]) == 1):
        res.append('')
    else:
        res.append(content[i][1])

### Accept the links for that redirects to the articles (Read more...portion)

In [601]:
links=[]
for my_tag in soup.find_all(class_="button button--smaller button--chromeless u-baseColor--buttonNormal"):
        links.append(my_tag.get('href'))

### Add the mising link in the 47th index

In [629]:
## add missing link
links.insert(47,'https://medium.com/@matrixainetwork')

In [531]:
import pandas as pd

## Convert all arrays into a Pandas Dataframe

In [631]:
data = pd.DataFrame({'1.Tag':'Artificial Intelligence','2.Name':name,'3.Title':title,'4.Body':body,'5.Upvotes':upvotes,'6.Date':date,'7.Comments':res,'8.Link':links}) 

In [719]:
data.head()

,1.Tag,2.Name,3.Title,4.Body,5.Upvotes,6.Date,7.Comments,8.Link
0,Artificial Intelligence,WiseWolf Fund,GAME-CHANGING TRENDS TO LOOK OUT FOR WITH AI,Artificial Intelligence is a state-of-the-art ...,12.9K,Apr 30,10 responses,https://medium.com/@wisewolf_fund/unique-trend...
1,Artificial Intelligence,Justin Lee,Chatbots were the next big thing: what happened?,"Oh, how the headlines blared:“…the 2016 bot",7.7K,Jun 5,41 responses,https://medium.com/swlh/chatbots-were-the-next...
2,Artificial Intelligence,Emmanuel Ameisen,Reinforcement Learning from scratch,Inspired by a great tutorial at O’Reilly AI,907,Jun 7,1 response,https://blog.insightdatascience.com/reinforcem...
3,Artificial Intelligence,Irhum Shafkat,Intuitively Understanding Convolutions for Dee...,Exploring the strong visual hierarchies that,1.92K,Jun 1,2 responses,https://towardsdatascience.com/intuitively-und...
4,Artificial Intelligence,Sam Drozdov,An intro to Machine Learning for designers,The basics of machine learning and how to appl...,2.3K,May 30,10 responses,https://uxdesign.cc/an-intro-to-machine-learni...


# Articles tagged in MACHINE LEARNING

In [710]:
driver = webdriver.Chrome('C:/Users/Sangarshanan Veera/Desktop/chromedriver.exe')
driver.get(machine_learning)

In [711]:
res = driver.execute_script("return document.documentElement.outerHTML")
driver.quit()

In [712]:
soup = BeautifulSoup(res, 'lxml')

## Repeat the process again for the ML tag

In [713]:
work=[]
for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link--darken link--accent u-accentColor--textNormal"):
    work.append(my_tag.text)
para=[]
for my_tag in soup.find_all(True,{'class':['graf graf--h4 graf-after--h3 graf--trailing graf--subtitle','graf graf--p graf-after--figure','graf graf--p graf-after--h3 graf--trailing']}):
    para.append(my_tag.text)
title=[]
for my_tag in soup.find_all(True,{'class':['graf graf--h3 graf-after--figure graf--title','graf graf--h3 graf--leading graf--title','graf graf--h3 graf-after--figure graf--trailing graf--title']}):
    title.append(my_tag.text)
read=[]
for my_tag in soup.find_all(class_="readingTime"):
    read.append(my_tag.get('title'))
upvotes=[]
for my_tag in soup.find_all('span',{'class':'u-textAlignCenter u-relative u-background js-actionMultirecommendCount u-marginLeft5'}):
    upvotes.append(my_tag.text)
date=[]
for my_tag in soup.find_all('time'):
    date.append(my_tag.text)
name = []
for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken"):
    name.append(my_tag.text)
content=[]
alllink=[]
for my_tag in soup.find_all(class_="postArticle postArticle--short js-postArticle js-trackedPost"):
        content.append(my_tag.text)
        for link in my_tag.find_all('a'):
            alllink.append(link.get('href'))
for i in range(0,len(content)):
    content[i]=content[i].replace(name[i],'')
    content[i]=content[i].replace(date[i],'')
    content[i]=content[i].replace(upvotes[i],'')
    content[i]=content[i].replace(title[i],'')
    content[i]=content[i].replace('…Read more…','*')
    content[i]=content[i].replace('Read more…','*')
    content[i]=content[i].replace('\xa0',' ')
    content[i]=content[i].replace(' in ', '')
    for tag in work:
        content[i] = content[i].replace(tag, '')
    content[i]=content[i].split('*')
body=[]
for i in range(0,len(content)):
    body.append(content[i][0])

res =[]
for i in range(0,len(content)):
    if (len(content[i]) == 1):
        res.append('')
    else:
        res.append(content[i][1])

links=[]
for my_tag in soup.find_all(class_="button button--smaller button--chromeless u-baseColor--buttonNormal"):
        links.append(my_tag.get('href'))

### No length issues...so directly include into pandas dataframe

In [714]:
data1 = pd.DataFrame({'1.Tag':'Machine Learning','2.Name':name,'3.Title':title,'4.Body':body,'5.Upvotes':upvotes,'6.Date':date,'7.Comments':res,'8.Link':links}) 

In [718]:
data1.head()

,1.Tag,2.Name,3.Title,4.Body,5.Upvotes,6.Date,7.Comments,8.Link
0,Machine Learning,Justin Lee,Chatbots were the next big thing: what happened?,"Oh, how the headlines blared:“…the 2016 bot",7.9K,Jun 5,42 responses,https://medium.com/swlh/chatbots-were-the-next...
1,Machine Learning,Conor Dewey,Python for Data Science: 8 Concepts You May Ha...,Mastering the little thingsPython,1.1K,Jun 5,5 responses,https://towardsdatascience.com/python-for-data...
2,Machine Learning,Emmanuel Ameisen,Reinforcement Learning from scratch,Inspired by a great tutorial at O’Reilly AI,912,Jun 7,2 responses,https://blog.insightdatascience.com/reinforcem...
3,Machine Learning,Irhum Shafkat,Intuitively Understanding Convolutions for Dee...,Exploring the strong visual hierarchies that,1.98K,Jun 1,2 responses,https://towardsdatascience.com/intuitively-und...
4,Machine Learning,Gant Laborde,Machine Learning: how to go from Zero to Hero,Start with “Why?” and end with “I’m ready!”,1.1K,Jun 6,2 responses,https://medium.freecodecamp.org/machine-learni...


# Articles tagged in DEEP LEARNING

In [650]:
driver = webdriver.Chrome('C:/Users/Sangarshanan Veera/Desktop/chromedriver.exe')
driver.get(deep_learning)

In [651]:
res = driver.execute_script("return document.documentElement.outerHTML")
driver.quit()

In [652]:
soup = BeautifulSoup(res, 'lxml')

In [704]:
work=[]
for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link--darken link--accent u-accentColor--textNormal"):
    work.append(my_tag.text)
para=[]
for my_tag in soup.find_all(True,{'class':['graf graf--h4 graf-after--h3 graf--trailing graf--subtitle','graf graf--p graf-after--figure','graf graf--p graf-after--h3 graf--trailing']}):
    para.append(my_tag.text)
title=[]
for my_tag in soup.find_all(True,{'class':['graf graf--h3 graf-after--figure graf--title','graf graf--h3 graf--leading graf--title','graf graf--h3 graf-after--figure graf--trailing graf--title']}):
    title.append(my_tag.text)
read=[]
for my_tag in soup.find_all(class_="readingTime"):
    read.append(my_tag.get('title'))
upvotes=[]
for my_tag in soup.find_all('span',{'class':'u-textAlignCenter u-relative u-background js-actionMultirecommendCount u-marginLeft5'}):
    upvotes.append(my_tag.text)
date=[]
for my_tag in soup.find_all('time'):
    date.append(my_tag.text)
name = []
for my_tag in soup.find_all(class_="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken"):
    name.append(my_tag.text)
content=[]
alllink=[]
for my_tag in soup.find_all(class_="postArticle postArticle--short js-postArticle js-trackedPost"):
        content.append(my_tag.text)
        for link in my_tag.find_all('a'):
            alllink.append(link.get('href'))


### Add the missing title in the 33rd index

In [705]:
title.insert(33,'“WTH does a neural network even learn?” — A newcomer’s dilemma')

In [706]:
for i in range(0,len(content)):
    content[i]=content[i].replace(name[i],'')
    content[i]=content[i].replace(date[i],'')
    content[i]=content[i].replace(upvotes[i],'')
    content[i]=content[i].replace(title[i],'')
    content[i]=content[i].replace('…Read more…','*')
    content[i]=content[i].replace('Read more…','*')
    content[i]=content[i].replace('\xa0',' ')
    content[i]=content[i].replace(' in ', '')
    for tag in work:
        content[i] = content[i].replace(tag, '')
    content[i]=content[i].split('*')
body=[]
for i in range(0,len(content)):
    body.append(content[i][0])

res =[]
for i in range(0,len(content)):
    if (len(content[i]) == 1):
        res.append('')
    else:
        res.append(content[i][1])

links=[]
for my_tag in soup.find_all(class_="button button--smaller button--chromeless u-baseColor--buttonNormal"):
        links.append(my_tag.get('href'))

In [707]:
data2 = pd.DataFrame({'1.Tag':'Deep Learning','2.Name':name,'3.Title':title,'4.Body':body,'5.Upvotes':upvotes,'6.Date':date,'7.Comments':res,'8.Link':links}) 

In [708]:
data2.head()

,1.Tag,2.Name,3.Title,4.Body,5.Upvotes,6.Date,7.Comments,8.Link
0,Deep Learning,Emmanuel Ameisen,Reinforcement Learning from scratch,Inspired by a great tutorial at O’Reilly AI,912,Jun 7,2 responses,https://blog.insightdatascience.com/reinforcem...
1,Deep Learning,Irhum Shafkat,Intuitively Understanding Convolutions for Dee...,Exploring the strong visual hierarchies that,1.98K,Jun 1,2 responses,https://towardsdatascience.com/intuitively-und...
2,Deep Learning,Gant Laborde,Machine Learning: how to go from Zero to Hero,Start with “Why?” and end with “I’m ready!”,1.1K,Jun 6,2 responses,https://medium.freecodecamp.org/machine-learni...
3,Deep Learning,Aman Dalmia,What I learned from interviewing at multiple A...,,2.3K,May 27,13 responses,https://blog.usejournal.com/what-i-learned-fro...
4,Deep Learning,Abhishek Parbhakar,Must know Information Theory concepts in Deep ...,,846,Jun 3,1 response,https://towardsdatascience.com/must-know-infor...


## Combine all three tags

In [720]:
f = data.append(data1, ignore_index=True)

In [722]:
alldata = f.append(data2,ignore_index=True)

In [724]:
alldata.shape

(185, 8)

## Convert to csv format

In [740]:
alldata.to_csv('medium.csv',encoding='utf-8')

In [741]:
df = pd.read_csv('medium.csv')

In [744]:
df.head()

,Unnamed: 0,1.Tag,2.Name,3.Title,4.Body,5.Upvotes,6.Date,7.Comments,8.Link
0,0,Artificial Intelligence,WiseWolf Fund,GAME-CHANGING TRENDS TO LOOK OUT FOR WITH AI,Artificial Intelligence is a state-of-the-art ...,12.9K,Apr 30,10 responses,https://medium.com/@wisewolf_fund/unique-trend...
1,1,Artificial Intelligence,Justin Lee,Chatbots were the next big thing: what happened?,"Oh, how the headlines blared:“…the 2016 bot",7.7K,Jun 5,41 responses,https://medium.com/swlh/chatbots-were-the-next...
2,2,Artificial Intelligence,Emmanuel Ameisen,Reinforcement Learning from scratch,Inspired by a great tutorial at O’Reilly AI,907,Jun 7,1 response,https://blog.insightdatascience.com/reinforcem...
3,3,Artificial Intelligence,Irhum Shafkat,Intuitively Understanding Convolutions for Dee...,Exploring the strong visual hierarchies that,1.92K,Jun 1,2 responses,https://towardsdatascience.com/intuitively-und...
4,4,Artificial Intelligence,Sam Drozdov,An intro to Machine Learning for designers,The basics of machine learning and how to appl...,2.3K,May 30,10 responses,https://uxdesign.cc/an-intro-to-machine-learni...
